<a href="https://colab.research.google.com/github/naz-mul94/CS69002_9A_18CS60R63/blob/master/nazmul.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Review Sentiment Analysis

In [0]:
import nltk

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Import Header files

In [0]:
import torch
import pandas as pd
import numpy as np
from string import punctuation
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

lemmatizer = WordNetLemmatizer()

In [0]:
url = 'https://github.com/binny-mathew/IITKGP_CS69002_Spring_2019/blob/master/Dataset/check.csv'
df1 = pd.read_csv(url, sep='\t')

In [11]:
from google.colab import files
uploaded = files.upload()

Saving Train_20K.csv to Train_20K.csv
Saving Test_5K.csv to Test_5K.csv


In [0]:
import io
df = pd.read_csv(open('Train_20K.csv', encoding='utf-8'), sep='\t')
df2 = pd.read_csv(open('Test_5K.csv', encoding='utf-8'), sep='\t')
#print (df2)

In [0]:
text_reviews_temp = df['text'].astype(str).tolist()

text_labels = df['label'].astype(int)

text_reviews = list()
#i = 1
sw = set(stopwords.words('english'))
for text_review in text_reviews_temp:
 # print (text_review)
  text_review = text_review.translate(str.maketrans('', '', punctuation))
  tokens = text_review.lower().split()
  #print (text_review)
  #print (tokens)
  #i=i+1
  
  
  terms = set(lemmatizer.lemmatize(term) for term in tokens)
  #print (terms)
  temp = terms.difference(sw)
  #print (temp)
  text_reviews.append(" ".join(temp))
  #print (text_reviews)
  #if i==2:
   # break;

In [0]:
test_reviews_temp = df2['text'].astype(str).tolist()
test_labels = df2['label'].astype(int)
test_reviews = list()
sw = set(stopwords.words('english'))
for test_review in test_reviews_temp:
  test_review = test_review.translate(str.maketrans('', '', punctuation))
  tokens = test_review.lower().split()
  terms = set(lemmatizer.lemmatize(term) for term in tokens)
  temp = terms.difference(sw)
  test_reviews.append(" ".join(temp))
  

In [0]:
def generate_word_ids(dataset):
  word_to_ix = {}
  count = {}
  #i=1
  for sent,_ in dataset:
      #if i==1:
       # i=i+1
        #print (sent)
      for word in sent.split():
          if word not in count:
            count[word] = 1
          else:
            count[word] = count[word] + 1
            if word not in word_to_ix:
              if count[word] >= 6:
                word_to_ix[word] = len(word_to_ix)
                
  return word_to_ix

In [0]:

data = tuple(zip(text_reviews,text_labels))

test_data = tuple(zip(test_reviews,test_labels))

In [17]:
word_to_ix = generate_word_ids(data+test_data)

VOCAB_SIZE = len(word_to_ix)
NUM_LABELS = 2
print(VOCAB_SIZE)

22887


#TASK1

## Intro to PyTorch

In [0]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)

**BE VERY CAREFUL WITH THE DIMENSIONS**

## Model Definition for the BOWClassifier

In [0]:
def make_bow_vector(sentence, word_to_ix):
    # create a vector of zeros of vocab size = len(word_to_idx)
    vec = torch.zeros(len(word_to_ix))
    for word in sentence.split():
        if word in word_to_ix:
            vec[word_to_ix[word]]+=1
    return vec.view(1, -1)

def make_target(label):
    return torch.LongTensor([label]).cuda()


#TASK1A

In [0]:
class Task1A(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task1A, self).__init__()
    self.lin = nn.Linear(vocab_size, 50)
    self.lin1 = nn.Linear(50, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin1(self.lin(x)))

In [0]:
Task_1A = Task1A(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask1A = nn.NLLLoss()
optTask1A = torch.optim.SGD(Task_1A.parameters(), lr = 0.001)

In [0]:
# Task1A
# the training loop
for epoch in range(3):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_1A.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_1A(bow_vec) # forward pass
        loss = loss_functionTask1A(probs, label)
        loss.backward()
        optTask1A.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_1A(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

In [44]:
torch.save(Task_1A,'Task1A.mdl')

from google.colab import files
files.download("Task1A.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task1A. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK1B

In [0]:
class Task1B(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task1B, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin1 = nn.Linear(100, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin1(self.lin(x)))

In [0]:
Task_1B = Task1B(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask1B = nn.NLLLoss()
optTask1B = torch.optim.SGD(Task_1B.parameters(), lr = 0.001)

In [50]:
# Task1B
# the training loop
for epoch in range(3):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_1B.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_1B(bow_vec) # forward pass
        loss = loss_functionTask1B(probs, label)
        loss.backward()
        optTask1B.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_1B(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


CURRENT LOSS: -0.8110874891281128
1
CURRENT LOSS: -0.9488620758056641
2
CURRENT LOSS: -0.9782378673553467
--- AFTER TRAINING ---
Precision: 0.8456860509312049
Recall: 0.8836378077839555
f1-score: 0.8642454845601089
Accuracy: 0.8602


In [51]:
torch.save(Task_1B,'Task1B.mdl')

from google.colab import files
files.download("Task1B.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task1B. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK1C

In [0]:
class Task1C(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task1C, self).__init__()
    self.lin = nn.Linear(vocab_size, 200)
    self.lin1 = nn.Linear(200, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin1(self.lin(x)))

In [0]:
Task_1C = Task1C(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask1C = nn.NLLLoss()
optTask1C = torch.optim.SGD(Task_1C.parameters(), lr = 0.005)

In [56]:
# Task1C
# the training loop
for epoch in range(3):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_1C.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_1C(bow_vec) # forward pass
        loss = loss_functionTask1C(probs, label)
        loss.backward()
        optTask1C.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_1C(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


CURRENT LOSS: -0.9914299845695496
1
CURRENT LOSS: -0.9968959093093872
2
CURRENT LOSS: -0.9987044334411621
--- AFTER TRAINING ---
Precision: 0.8769539078156313
Recall: 0.868943606036537
f1-score: 0.8729303810093758
Accuracy: 0.8726


In [57]:
torch.save(Task_1C,'Task1C.mdl')

from google.colab import files
files.download("Task1C.mdl")

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:251: UserWarning: Couldn't retrieve source code for container of type Task1C. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


#TASK1D

In [0]:
class Task1D(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task1D, self).__init__()
    self.lin = nn.Linear(vocab_size, 500)
    self.lin1 = nn.Linear(500, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin1(self.lin(x)))

In [0]:
Task_1D = Task1D(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
# define a loss function and an optimizer
loss_functionTask1D = nn.NLLLoss()
optTask1D = torch.optim.SGD(Task_1D.parameters(), lr = 0.005)

In [0]:
# Task1D
# the training loop
for epoch in range(3):
    print(epoch)
    for instance, label in data:
        # get the training data
        Task_1D.zero_grad()
        bow_vec = Variable(make_bow_vector(instance, word_to_ix)).cuda()
        label = Variable(make_target(label))
        probs = Task_1D(bow_vec) # forward pass
        loss = loss_functionTask1D(probs, label)
        loss.backward()
        optTask1D.step()
    print('CURRENT LOSS: {}'.format(loss.data))
    
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

print('--- AFTER TRAINING ---')
predictions = list()
for instance, label in test_data:
    bow_vec = Variable(make_bow_vector(instance, word_to_ix))
    logprobs = Task_1D(bow_vec.cuda())
    pred = np.argmax(logprobs.data.cpu().numpy())
    predictions.append(pred)
    
print('Precision: {}'.format(precision_score(test_labels,predictions)))
print('Recall: {}'.format(recall_score(test_labels,predictions)))
print('f1-score: {}'.format(f1_score(test_labels,predictions)))
print('Accuracy: {}'.format(accuracy_score(test_labels,predictions)))

In [0]:
torch.save(Task_1D,'Task1D.mdl')

from google.colab import files
files.download("Task1D.mdl")

#TASK2A

In [0]:
class Task2A(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task1B, self).__init__()
    self.lin = nn.Linear(vocab_size, 10)
    self.lin1 = nn.Linear(10, 10)
    self.lin2 = nn.Linear(10, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin2(self.lin1(self.lin(x))))

In [0]:
class Task2B(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task2B, self).__init__()
    self.lin = nn.Linear(vocab_size, 20)
    self.lin1 = nn.Linear(20, 10)
    self.lin2 = nn.Linear(10, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin2(self.lin1(self.lin(x))))

In [0]:
class Task3B(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task3B, self).__init__()
    self.lin = nn.Linear(vocab_size, 30)
    self.lin1 = nn.Linear(30, 30)
    self.lin2 = nn.Linear(30, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin2(self.lin1(self.lin(x))))

In [0]:
class Task4B(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task4B, self).__init__()
    self.lin = nn.Linear(vocab_size, 50)
    self.lin1 = nn.Linear(50, 50)
    self.lin2 = nn.Linear(50, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin2(self.lin1(self.lin(x))))

In [0]:
class Task5B(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task5B, self).__init__()
    self.lin = nn.Linear(vocab_size, 50)
    self.lin1 = nn.Linear(50, 100)
    self.lin2 = nn.Linear(100, num_labels)
    
  def forward(self, x):
    return F.softmax(self.lin2(self.lin1(self.lin(x))))

In [0]:
class Task1C(nn.Module):
  def __init__(self, num_labels, vocab_size):
    super(Task1C, self).__init__()
    self.lin = nn.Linear(vocab_size, 100)
    self.lin1 = nn.Linear(100, 50)
    self.lin2 = nn.Linear(50, 10)
    self.lin3 = nn.Linear(10, num_labels)
    
    
  def forward(self, x):
    return F.softmax(self.lin3(self.lin2(self.lin1(self.lin(x)))))

##Generate the BOW Vectors

In [0]:
Task_1B = Task1B(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
Task_2B = Task2B(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
Task_3B = Task3B(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
Task_4B = Task4B(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
Task_5B = Task5B(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
Task_1C = Task1C(NUM_LABELS, VOCAB_SIZE).cuda()

In [0]:
for param in bow.parameters():
    print(param,param.size())

### Let's check with a sample case how our model is working

In [0]:
# define a loss function and an optimizer
loss_function = nn.NLLLoss()
opt = torch.optim.SGD(bow.parameters(), lr = 0.1)

In [0]:
# define a loss function and an optimizer
loss_functionTask1B = nn.NLLLoss()
optTask1B = torch.optim.SGD(Task1B.parameters(), lr = 0.006)

In [0]:
# define a loss function and an optimizer
loss_functionTask2B = nn.NLLLoss()
optTask2B = torch.optim.SGD(Task2B.parameters(), lr = 0.006)

In [0]:
# define a loss function and an optimizer
loss_functionTask3B = nn.NLLLoss()
optTask3B = torch.optim.SGD(Task3B.parameters(), lr = 0.01)

In [0]:
# define a loss function and an optimizer
loss_functionTask4B = nn.NLLLoss()
optTask4B = torch.optim.SGD(Task4B.parameters(), lr = 0.001)

In [0]:
# define a loss function and an optimizer
loss_functionTask5B = nn.NLLLoss()
optTask5B = torch.optim.SGD(Task5B.parameters(), lr = 0.001)

In [0]:
# define a loss function and an optimizer
loss_functionTask1C = nn.NLLLoss()
optTask1C = torch.optim.SGD(Task1C.parameters(), lr = 0.001)

In [0]:
from google.colab import files
temp_test = files.upload()


In [0]:
torch.load(io.BytesIO(temp_test['model.mdl']))